In [ ]:
!nvidia-smi #to check gpu

Mon Apr 22 16:29:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

##Installing and importing dependencies

In [ ]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 12.3 MB/s eta 0:00:00


In [ ]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay

In [ ]:
model_checkpoint='Helsinki-NLP/opus-mt-hi-en'

Model - https://huggingface.co/Helsinki-NLP/opus-mt-en-hi

Dataset used for fine tuning- https://huggingface.co/datasets/cfilt/iitb-english-hindi

In [ ]:
raw_dataset=load_dataset('cfilt/iitb-english-hindi')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [ ]:
raw_dataset #metadata

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [ ]:
raw_dataset['train'][109]

{'translation': {'en': 'Validates application accessibility',
  'hi': 'अनुप्रयोग की पहुंचनीयता का सत्यापन करता है'}}

##Preprocessing


In [ ]:
#to create the embeddings
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
tokenizer(["Hi, My name is Gayathri is Gayathri","This is my AIRIW Project"])

{'input_ids': [[7625, 2, 558, 296, 25, 2476, 277, 667, 11477, 3152, 25, 2476, 277, 667, 11477, 3152, 0], [273, 25, 140, 242, 12508, 382, 2875, 9757, 8183, 867, 162, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer(): #in the target language
  print(tokenizer(["अनुप्रयोग की पहुंचनीयता का सत्यापन करता है"]))

{'input_ids': [[50, 2267, 493, 3262, 2553, 1100, 582, 3488, 832, 2383, 50, 395, 123, 50, 2553, 2207, 3262, 552, 4690, 493, 123, 3488, 417, 239, 50, 395, 239, 50, 818, 417, 1100, 3488, 239, 2553, 493, 50, 395, 582, 417, 239, 50, 2207, 8492, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
max_input_length =128
max_target_length=128

source_lang="hi"
target_lang="en"

def preprocess_function(examples):
  inputs=[ex[source_lang] for ex in examples['translation']]
  targets=[ex[target_lang] for ex in examples['translation']]
  model_inputs=tokenizer(inputs,max_length=max_input_length,truncation=True)

  with tokenizer.as_target_tokenizer():
    labels=tokenizer(targets,max_length=max_target_length,truncation=True)
  model_inputs["labels"]=labels["input_ids"]
  return model_inputs


In [ ]:
preprocess_function(raw_dataset["train"][:2])

{'input_ids': [[62, 4414, 21, 8765, 13268, 488, 22086, 30, 2036, 824, 0], [5893, 178, 9705, 18638, 6077, 8765, 13268, 488, 12832, 2606, 9062, 395, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[3564, 78, 4315, 138, 47442, 333, 4373, 0], [15745, 25117, 5755, 23, 564, 50, 52610, 48982, 871, 0]]}

In [ ]:
tokenized_datasets=raw_dataset.map(preprocess_function,batched=True)

Map:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [ ]:
model=TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/305M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-hi-en.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
batch_size =16
learning_rate=2e-5
weight_decay=0.01
num_train_epochs=10


In [ ]:
data_collator=DataCollatorForSeq2Seq(tokenizer,model=model,return_tensors="tf")


In [ ]:
generation_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model,return_tensors="tf",pad_to_multiple_of=128)



In [ ]:
train_dataset=model.prepare_tf_dataset(
    tokenized_datasets["test"], #using test here because the train size if huge change it when you have better wifi lol
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [ ]:
validation_dataset=model.prepare_tf_dataset(
    tokenized_datasets["validation"], #using test here because the train size if huge change it when you have better wifi lol
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [ ]:
generation_dataset=model.prepare_tf_dataset(
    tokenized_datasets["validation"], #using test here because the train size if huge change it when you have better wifi lol
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [ ]:
optimizer=AdamWeightDecay(learning_rate=learning_rate,weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_dataset,validation_data=validation_dataset,epochs=1)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
156/156 [==============================] - 147s 385ms/step - loss: 3.5156 - val_loss: 3.4966


In [ ]:
model.save_pretrained("tf_model/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[61126]], 'forced_eos_token_id': 0}


##Testing

In [ ]:
input_text="कितने वर्षों के दौरान जीडिपी की वृद्धि हुई? और फिर मुझे बताओ कि अपनी कंपनी के साथ आपकी योजना क्या है"
tokenized=tokenizer([input_text],return_tensors='np')
out=model.generate(**tokenized,max_length=128)
print(out)

tf.Tensor(
[[61126  4302     4   363     4  2476  1182  1399  3825    10   194  1028
    143   104    78  4195    25    48    78  3753     3     0 61126 61126]], shape=(1, 24), dtype=int32)


In [ ]:
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0],skip_special_tokens=True))

Over the years the GDP increased and then tell me what your plan is with your company.


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write)

In [ ]:
!git config --global credential.helper store

In [ ]:
model.push_to_hub("HI-EN-translation")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[61126]], 'forced_eos_token_id': 0}


tf_model.h5:   0%|          | 0.00/304M [00:00<?, ?B/s]

In [ ]:
tokenizer.push_to_hub("HI-EN-translation")

README.md:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Nettem-Gayathri/HI-EN-translation/commit/865a2a066e3bd7a5ab5d247fd22e0aa5dabfcdc3', commit_message='Upload tokenizer', commit_description='', oid='865a2a066e3bd7a5ab5d247fd22e0aa5dabfcdc3', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Nettem-Gayathri/EN-HI-translation")
model = AutoModelForSeq2SeqLM.from_pretrained("Nettem-Gayathri/EN-HI-translation",from_tf=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All TF 2.0 model weights were used when initializing MarianMTModel.

Some weights of MarianMTModel were not initialized from the TF 2.0 model and are newly initialized: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
text = "Information has surfaced in recent years suggesting that Julius Rosenberg was involved in passing some form of intelligence to Soviet officials during the Second World War."
text="What is the article about?"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs)

translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Translated Text:", translated_text)

Translated Text: इस लेख के बारे में क्या कहा जा सकता है?


In [ ]:
text="What is this about"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs)

translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Translated Text:", translated_text)

Translated Text: इस बारे में क्या है


In [ ]:

text="What is being said about the Electoral Bond Scheme?"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs)

translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Translated Text:", translated_text)

Translated Text: ई-क्लीय बॉन्ड योजना के बारे में क्या कहा जा सकता है?


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer1 = AutoTokenizer.from_pretrained("Nettem-Gayathri/HI-EN-translation")
model1 = AutoModelForSeq2SeqLM.from_pretrained("Nettem-Gayathri/HI-EN-translation",from_tf=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All TF 2.0 model weights were used when initializing MarianMTModel.

Some weights of MarianMTModel were not initialized from the TF 2.0 model and are newly initialized: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
text = "प्रधानमंत्री ने कहा कि चुनावीय सम्बन्ध योजना को सफल कहानी भी समझा जाना चाहिए क्योंकि इसने दिखाया है कि राजनीतिक पार्टियों के लिए अंशदान किया था.उन्होंने यह भी कहा कि योजना में सुधार का बहुत विस्तार था"
inputs = tokenizer1(text, return_tensors="pt")
outputs = model1.generate(**inputs)

translated_text = tokenizer1.decode(outputs[0], skip_special_tokens=True)
print("Translated Text:", translated_text)

Translated Text: The prime minister said that the election relationship plan should also be considered a successful story because it showed that it had contributed for political parties.
